In [1]:
from datetime import date, datetime, timedelta
import time
import dateutil
from dateutil import tz

import json

import pandas as pd 
import itertools
from itertools import product

import numpy as np
import urllib.request, json 
import re

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
import requests
from pandas import DataFrame
import csv

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

import os
import glob
import ast 

from time import sleep
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import xmltodict 
import lxml
from pretty_html_table import build_table

import json
import requests

import xml.etree.ElementTree as et 

import simpleaudio as sa

import dataframe_image as dfi

In [2]:
#Game Spreads  / Moneyline Pull 
# Documentation: http://xmlfeed.intertops.com/XMLOddsFeed/#

#####api_token = '641af124-9174-eb11-8ad8-003048dd52d5'

#?sportId=5 = Basketball (NBA / NCAAB) 
#?sportId=16 = Hockey

In [3]:
iterabledf = []
iterabledf = pd.DataFrame(columns=["Team 1","Team 2", "Team 1 Money Line", "Team 2 Money Line","Sport","Time Pulled (CST)"])
iterabledf = iterabledf.set_index(['Team 1','Team 2','Sport','Time Pulled (CST)'])

profitablelog = []
profitablelog = pd.DataFrame(columns=["Team 1","Team 2","Sport","Time Pulled (CST)","Gambit Prev Reward 1",
"Gambit Prev Reward 2",
"Gambit Curr Reward 1",
"Gambit Curr Reward 2",
"Tokens to Hedge",
"Favored Prev - Underdog Curr Profit %",
"Underdog Prev - Favored Curr Profit %",
"Favored Prev - Underdog Curr Profit % Favored Bet",
"Favored Prev - Underdog Curr Profit % Underdog Bet",
"Underdog Prev - Favored Curr Profit % Underdog Bet",
"Underdog Prev - Favored Curr Profit % Favored Bet"])
profitablelog = profitablelog.set_index(["Team 1","Team 2","Sport", "Time Pulled (CST)"])

TotalTime = 0



In [4]:
#############################################################################################################
# Gambit
#############################################################################################################
start_date = datetime.now()

GambitProfit = requests.get("https://api.gambitprofit.com/gambit-plays?_limit=200&_sort=PlayDate:DESC").json()
GambitProfitAPI = pd.DataFrame.from_dict(json.loads(json.dumps(GambitProfit)))[['PlayDate','Team1','Team2',"Calc"]]

GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team1'].astype(str)
GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team2'].astype(str)

GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team 1 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()
GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team 2 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()

GambitProfitAPI['PlayDate'] = pd.to_datetime(GambitProfitAPI['PlayDate'], 
                                       format='%Y-%m-%dT%H:%M:%S.000Z', 
                                       errors='coerce') - timedelta(hours=6, minutes=0)

GambitProfitAPI = GambitProfitAPI.drop(columns=['Team1','Team2'])
after_start_date = GambitProfitAPI["PlayDate"] >= start_date
GambitProfitAPI = GambitProfitAPI.loc[after_start_date]

GambitProfitAPI["Calc"] = GambitProfitAPI["Calc"].astype(str)
GambitProfitAPI["Profitable"] = GambitProfitAPI["Calc"].str.extract(r"Profitable': (.{3})")[0]         

GambitProfitAPI = GambitProfitAPI.drop(columns=['Calc','Profitable'])

GambitProfitAPI['Source'] = "Gambit"

Team1List = list(GambitProfitAPI['Team 1 Name'])
Team2List = list(GambitProfitAPI['Team 2 Name'])
TotalList = Team1List + Team2List
TotalList = [x for x in TotalList if str(x) != 'nan']

#############################################################################################################
# Intertops
#############################################################################################################

urldelta = '&delta=1440'

#############################################################################################################
# Basketbell
#############################################################################################################

######################################################################################
# NBA Odds - Moneyline - Single Match
######################################################################################
basketball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=5' + urldelta)
bs_content = BeautifulSoup(basketball.content, "lxml")
nba = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NBA Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

teams = []
odds = []

for m in nba:
    for line in m:
        teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
        odds.append(re.findall('o="(.*?)"', str(m)))

teams = teams[0]
odds = odds[0]

N = 2
teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

nbaoddsdf = pd.concat([teamdf, oddsdf], axis=1)
nbaoddsdf['Sport'] = "NBA"    

######################################################################################
# NCAAB - Moneyline - Single Match
######################################################################################
bs_content = BeautifulSoup(basketball.content, "lxml")
ncaab = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NCAAB Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

teams = []
odds = []

for m in ncaab:
    for line in m:
        teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
        odds.append(re.findall('o="(.*?)"', str(m)))

teams = teams[0]
odds = odds[0]

N = 2
teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

ncaaboddsdf = pd.concat([teamdf, oddsdf], axis=1)
ncaaboddsdf['Sport'] = "NCAAB"    
 
#############################################################################################################
# Hockey
#############################################################################################################

######################################################################################
# NHL Odds - Moneyline - Single Match
######################################################################################
icehockey = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=16'+ urldelta)
bs_content = BeautifulSoup(icehockey.content, "lxml")
nhl = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NHL Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

teams = []
odds = []

for m in nhl:
    for line in m:
        teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
        odds.append(re.findall('o="(.*?)"', str(m)))

teams = teams[0]
odds = odds[0]

N = 2
teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

nhloddsdf = pd.concat([teamdf, oddsdf], axis=1)
nhloddsdf['Sport'] = "NHL"    



#############################################################################################################
# Baseball
#############################################################################################################

######################################################################################
# MLB Odds 'Spring Training' - Moneyline - Single Match
######################################################################################
baseball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=4' + urldelta)
bs_content = BeautifulSoup(baseball.content, "lxml")
mlbst = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"MLB Spring Training"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

teams = []
odds = []

for m in mlbst:
    for line in m:
        teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
        odds.append(re.findall('o="(.*?)"', str(m)))

teams = teams[0]
odds = odds[0]

N = 2
teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

mlbstoddsdf = pd.concat([teamdf, oddsdf], axis=1)
mlbstoddsdf['Sport'] = "MLB Spring Training"   



######################################################################################
# Consolidate and Append the Old Table
######################################################################################
allsportsoddsdf = nbaoddsdf.append([ncaaboddsdf, nhloddsdf,mlbstoddsdf])
allsportsoddsdf["Time Pulled (CST)"] = pd.Series([datetime.now()] * len(allsportsoddsdf),dtype=pd.StringDtype())
allsportsoddsdf["Team 1 Money Line"] = allsportsoddsdf["Team 1 Money Line"].astype(float)
allsportsoddsdf["Team 2 Money Line"] = allsportsoddsdf["Team 2 Money Line"].astype(float)

######################################################################################
# Filtering ESPN / Dratings Table by Gambit API
######################################################################################

allsportsoddsdf['Team 1'] = allsportsoddsdf['Team 1'].str.replace('&amp;','&')
allsportsoddsdf['Team 2'] = allsportsoddsdf['Team 2'].str.replace('&amp;','&')

allsportsoddsdf['Team 1 in API'] = allsportsoddsdf['Team 1'].apply(lambda x: ','.join([i for i in TotalList if i in x]))
allsportsoddsdf['Team 2 in API'] = allsportsoddsdf['Team 2'].apply(lambda x: ','.join([i for i in TotalList if i in x]))

allsportsoddsdf = allsportsoddsdf[(allsportsoddsdf['Team 1 in API'] != '') | (allsportsoddsdf['Team 2 in API'] != '')]
allsportsoddsdf = allsportsoddsdf.drop(columns=['Team 1 in API','Team 2 in API'])

######################################################################################
# Appending the New Data to the Old Data 
######################################################################################
allsportsoddsdf = allsportsoddsdf.set_index(['Team 1','Team 2','Sport','Time Pulled (CST)'])
iterabledf = iterabledf.append(allsportsoddsdf,sort=True)
iterabledf = iterabledf.sort_index(ascending=True)

# print(allsportsoddsdf,'\n',iterabledf)
iterabledf

,,,,Team 1 Money Line,Team 2 Money Line
Team 1,Team 2,Sport,Time Pulled (CST),,
Edmonton Oilers,Calgary Flames,NHL,2021-03-15 17:30:34.464268,2.0500,1.8000
Indiana Pacers,Denver Nuggets,NBA,2021-03-15 17:30:34.464268,2.6000,1.5556
Los Angeles Clippers,Dallas Mavericks,NBA,2021-03-15 17:30:34.464268,1.8000,2.0500
Los Angeles Lakers,Golden State Warriors,NBA,2021-03-15 17:30:34.464268,1.8000,2.0500
Memphis Grizzlies,Phoenix Suns,NBA,2021-03-15 17:30:34.464268,3.3500,1.3636
Montreal Canadiens,Winnipeg Jets,NHL,2021-03-15 17:30:34.464268,1.8696,1.9524
New York Knicks,Brooklyn Nets,NBA,2021-03-15 17:30:34.464268,3.8000,1.2857
San Antonio Spurs,Detroit Pistons,NBA,2021-03-15 17:30:34.464268,1.7692,2.1000
San Jose Sharks,Vegas Golden Knights,NHL,2021-03-15 17:30:34.464268,2.8500,1.4444


In [ ]:
for x in range(0,9999999999999999999999999999999999999999999999999999999999999999999999999999999999999999):    
    print(iterabledf)
    try:
        for x in range(0,99999999999999999999999999999999999999999999999999999999999999999999999999999):    
            if x % 120 == 0:
                print("We Are On Iteration: ", x)
            
            start_time = time.time()
            start_date = datetime.now()        
                
            #############################################################################################################
            # Gambit
            #############################################################################################################

            GambitProfit = requests.get("https://api.gambitprofit.com/gambit-plays?_limit=200&_sort=PlayDate:DESC").json()
            GambitProfitAPI = pd.DataFrame.from_dict(json.loads(json.dumps(GambitProfit)))[['PlayDate','Team1','Team2',"Calc"]]

            GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team1'].astype(str)
            GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team2'].astype(str)

            GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team 1 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()
            GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team 2 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()

            GambitProfitAPI['PlayDate'] = pd.to_datetime(GambitProfitAPI['PlayDate'], 
                                                   format='%Y-%m-%dT%H:%M:%S.000Z', 
                                                   errors='coerce') - timedelta(hours=6, minutes=0)

            GambitProfitAPI = GambitProfitAPI.drop(columns=['Team1','Team2'])
            after_start_date = GambitProfitAPI["PlayDate"] >= start_date
            GambitProfitAPI = GambitProfitAPI.loc[after_start_date]

            GambitProfitAPI["Calc"] = GambitProfitAPI["Calc"].astype(str)
            GambitProfitAPI["Profitable"] = GambitProfitAPI["Calc"].str.extract(r"Profitable': (.{3})")[0]         

            GambitProfitAPI = GambitProfitAPI.drop(columns=['Calc','Profitable'])

            GambitProfitAPI['Source'] = "Gambit"

            Team1List = list(GambitProfitAPI['Team 1 Name'])
            Team2List = list(GambitProfitAPI['Team 2 Name'])
            TotalList = Team1List + Team2List
            TotalList = [x for x in TotalList if str(x) != 'nan']


            #############################################################################################################
            # Intertops
            #############################################################################################################
            
            #initialize timer
            #initialize for setting different request lookback periods
            urldelta = '&delta=1'

         #############################################################################################################
            # Basketbell
         #############################################################################################################

            ######################################################################################
            # NBA Odds - Moneyline - Single Match
            ######################################################################################
            basketball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=5' + urldelta)
            bs_content = BeautifulSoup(basketball.content, "lxml")
            nba = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NBA Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

            teams = []
            odds = []

            for m in nba:
                for line in m:
                    teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                    odds.append(re.findall('o="(.*?)"', str(m)))

            teams = teams[0]
            odds = odds[0]

            N = 2
            teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
            teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

            oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
            oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

            nbaoddsdf = pd.concat([teamdf, oddsdf], axis=1)
            nbaoddsdf['Sport'] = "NBA"    


            ######################################################################################
            # NCAAB - Moneyline - Single Match
            ######################################################################################
            bs_content = BeautifulSoup(basketball.content, "lxml")
            ncaab = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NCAAB Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

            teams = []
            odds = []

            for m in ncaab:
                for line in m:
                    teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                    odds.append(re.findall('o="(.*?)"', str(m)))

            teams = teams[0]
            odds = odds[0]

            N = 2
            teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
            teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

            oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
            oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

            ncaaboddsdf = pd.concat([teamdf, oddsdf], axis=1)
            ncaaboddsdf['Sport'] = "NCAAB"    

         #############################################################################################################
            # Hockey
         #############################################################################################################

            ######################################################################################
            # NHL Odds - Moneyline - Single Match
            ######################################################################################
            icehockey = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=16'+ urldelta)
            bs_content = BeautifulSoup(icehockey.content, "lxml")
            nhl = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NHL Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

            teams = []
            odds = []

            for m in nhl:
                for line in m:
                    teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                    odds.append(re.findall('o="(.*?)"', str(m)))

            teams = teams[0]
            odds = odds[0]

            N = 2
            teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
            teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

            oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
            oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

            nhloddsdf = pd.concat([teamdf, oddsdf], axis=1)
            nhloddsdf['Sport'] = "NHL"    
            
            
            #############################################################################################################
            # Baseball
            #############################################################################################################

            ######################################################################################
            # MLB Odds 'Spring Training' - Moneyline - Single Match
            ######################################################################################
            baseball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=4' + urldelta)
            bs_content = BeautifulSoup(baseball.content, "lxml")
            mlbst = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"MLB Spring Training"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

            teams = []
            odds = []

            for m in mlbst:
                for line in m:
                    teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                    odds.append(re.findall('o="(.*?)"', str(m)))

            teams = teams[0]
            odds = odds[0]

            N = 2
            teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
            teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

            oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
            oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

            mlbstoddsdf = pd.concat([teamdf, oddsdf], axis=1)
            mlbstoddsdf['Sport'] = "MLB Spring Training"      
        
            
            ######################################################################################
            # Consolidate and Append the Old Table
            ######################################################################################
            allsportsoddsdf = nbaoddsdf.append([ncaaboddsdf, nhloddsdf,mlbstoddsdf])
            allsportsoddsdf["Time Pulled (CST)"] = pd.Series([datetime.now()] * len(allsportsoddsdf),dtype=pd.StringDtype())
            allsportsoddsdf["Team 1 Money Line"] = allsportsoddsdf["Team 1 Money Line"].astype(float)
            allsportsoddsdf["Team 2 Money Line"] = allsportsoddsdf["Team 2 Money Line"].astype(float)

            ######################################################################################
            # Filtering ESPN / Dratings Table by Gambit API
            ######################################################################################

            allsportsoddsdf['Team 1'] = allsportsoddsdf['Team 1'].str.replace('&amp;','&')
            allsportsoddsdf['Team 2'] = allsportsoddsdf['Team 2'].str.replace('&amp;','&')

            allsportsoddsdf['Team 1 in API'] = allsportsoddsdf['Team 1'].apply(lambda x: ','.join([i for i in TotalList if i in x]))
            allsportsoddsdf['Team 2 in API'] = allsportsoddsdf['Team 2'].apply(lambda x: ','.join([i for i in TotalList if i in x]))

            allsportsoddsdf = allsportsoddsdf[(allsportsoddsdf['Team 1 in API'] != '') | (allsportsoddsdf['Team 2 in API'] != '')]
            allsportsoddsdf = allsportsoddsdf.drop(columns=['Team 1 in API','Team 2 in API'])

            ######################################################################################
            # Appending the New Data to the Old Data 
            ######################################################################################
            allsportsoddsdf = allsportsoddsdf.set_index(['Team 1','Team 2','Sport','Time Pulled (CST)'])
            iterabledf = iterabledf.append(allsportsoddsdf,sort=True)
            iterabledf = iterabledf.sort_index(ascending=True)

            ######################################################################################
            # Calculating the Difference in the two Records
            ######################################################################################    
            #Duplicating the Table to Create the Diffs Column
            df = iterabledf

            df['diffs'] = np.nan
            df['diffs2'] = np.nan

            df['diffs'] = df.groupby(['Team 1','Team 2','Sport'])["Team 1 Money Line"].diff()
            df['diffs2'] = df.groupby(['Team 1','Team 2','Sport'])["Team 2 Money Line"].diff()

            df = df[(df["diffs"] > 0) |  (df["diffs"] < 0)]
            

            #dfTF = df[(df["diffs"] > 0) |  (df["diffs"] < 0)]
            #df = df.loc[dfTF]

            df["Prev Team 1 Money Line"] = df["Team 1 Money Line"] - df["diffs"]
            df["Prev Team 2 Money Line"] = df["Team 2 Money Line"] - df["diffs2"]
            
            Int = 0
            Coef = 1

#             Int = 0.0143989291341833
#             Coef = 0.98528641137416

        
            df["Gambit Prev Reward 1"] = round((round(df["Prev Team 1 Money Line"],2)*Coef)+Int,3)
            df["Gambit Prev Reward 2"] = round((round(df["Prev Team 2 Money Line"],2)*Coef)+Int,3)
            df["Gambit Curr Reward 1"] = round((round(df["Team 1 Money Line"],2)*Coef)+Int,3)
            df["Gambit Curr Reward 2"] = round((round(df["Team 2 Money Line"],2)*Coef)+Int,3)

            ######################################################################################
            # IF the DF Is Not Empty (Meaning There Was a Change in the Odds)
            ######################################################################################    
            if not df.empty:        
                
                PublishClean = df.drop(columns=['Team 1 Money Line','Team 2 Money Line','diffs','diffs2', 'Prev Team 2 Money Line','Prev Team 1 Money Line'])
                dfi.export(PublishClean.T, '/Users/lolatlogan/Downloads/SB/MoneyLineChanges/'+str(datetime.now())+'.png')

                testdf = df

                testdf["Tokens to Hedge"] = int(round(int(10000),0))
                testdf["Tokens to Hedge"] = testdf["Tokens to Hedge"].astype(int)
                
                profitcuttoff = 3.5
                
                #Favored Prev * Under Dog Curr
                testdf["Favored Prev - Underdog Curr Profit %"] = round((((((testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].min(axis=1)*testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].max(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].min(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].max(axis=1)))+.05)-1)*100) / .95,3)

                #Underdog Prev * Favored Curr
                testdf["Underdog Prev - Favored Curr Profit %"] = round((((((testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].max(axis=1)*testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].min(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].max(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].min(axis=1)))+.05)-1)*100)  / .95,3)
        
                #Favored Prev Bet = Underdog Curr / (Favored Prev + Underdog Curr)
                testdf["Favored Prev - Underdog Curr Profit % Favored Bet"] =  int(round(((testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].max(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].min(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].max(axis=1))) * testdf["Tokens to Hedge"],0))
                testdf["Favored Prev - Underdog Curr Profit % Favored Bet"] = testdf["Favored Prev - Underdog Curr Profit % Favored Bet"].astype(int)
                #Underdog Curr Bet = Favored Prev / (Favored Prev + Underdog Curr)
                testdf["Favored Prev - Underdog Curr Profit % Underdog Bet"] = int(round(((testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].min(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].min(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].max(axis=1))) * testdf["Tokens to Hedge"],0))
                testdf["Favored Prev - Underdog Curr Profit % Underdog Bet"] = testdf["Favored Prev - Underdog Curr Profit % Underdog Bet"].astype(int)
                
                #Underdog Prev Bet = Favored Curr / (Underdog Prev + Favored Curr)
                testdf["Underdog Prev - Favored Curr Profit % Underdog Bet"] = int(round(((testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].min(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].max(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].min(axis=1))) * testdf["Tokens to Hedge"],0))
                testdf["Underdog Prev - Favored Curr Profit % Underdog Bet"] = testdf["Underdog Prev - Favored Curr Profit % Underdog Bet"].astype(int)
                #Favored Curr Bet = Underdog Prev / (Underdog Prev + Favored Curr)
                testdf["Underdog Prev - Favored Curr Profit % Favored Bet"] =  int(round(((testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].max(axis=1)) / (testdf[['Gambit Prev Reward 1','Gambit Prev Reward 2']].max(axis=1)+testdf[['Gambit Curr Reward 1','Gambit Curr Reward 2']].min(axis=1))) * testdf["Tokens to Hedge"],0))
                testdf["Underdog Prev - Favored Curr Profit % Favored Bet"] =  testdf["Underdog Prev - Favored Curr Profit % Favored Bet"].astype(int)
                
                clean = testdf

                clean = clean.drop(columns=['Team 1 Money Line',"Team 2 Money Line","diffs","diffs2","Prev Team 1 Money Line","Prev Team 2 Money Line"])
                clean = clean[(clean["Favored Prev - Underdog Curr Profit %"] > profitcuttoff) | (clean["Underdog Prev - Favored Curr Profit %"] > profitcuttoff)] 
                
                clean = clean[["Gambit Prev Reward 1","Gambit Prev Reward 2","Gambit Curr Reward 1","Gambit Curr Reward 2","Tokens to Hedge","Favored Prev - Underdog Curr Profit %","Favored Prev - Underdog Curr Profit % Favored Bet","Favored Prev - Underdog Curr Profit % Underdog Bet","Underdog Prev - Favored Curr Profit %","Underdog Prev - Favored Curr Profit % Underdog Bet","Underdog Prev - Favored Curr Profit % Favored Bet"]]
                
                if not clean.empty:
                    sa.WaveObject.from_wave_file('/Users/lolatlogan/Downloads/mixkit-air-in-a-hit-2161.wav').play()

                    profitable = clean
                    profitable = profitablelog.append(profitable,sort=True)
                    profitable.to_excel("/Users/lolatlogan/Downloads/profitablelog.xlsx")  
                    
                    print(clean) 

                    if sum(clean["Favored Prev - Underdog Curr Profit %"]) > sum(clean["Underdog Prev - Favored Curr Profit %"]):
                        clean = clean.drop(columns=['Underdog Prev - Favored Curr Profit %','Underdog Prev - Favored Curr Profit % Underdog Bet','Underdog Prev - Favored Curr Profit % Favored Bet'])
                        Favored_Prev_Underdog_Curr_Profit = clean
                        
                        print(Favored_Prev_Underdog_Curr_Profit)                                                

                        dfi.export(Favored_Prev_Underdog_Curr_Profit.T, '/Users/lolatlogan/Downloads/SB/Backdoor Hedging/'+str(datetime.now())+'.png')
                        
                        Html_file = open("/Users/lolatlogan/Public/index.html","w")
                        Html_file.write(str(Favored_Prev_Underdog_Curr_Profit.to_html()))
                        Html_file.close()
                        ! firebase deploy
                        
                    elif sum(clean["Underdog Prev - Favored Curr Profit %"]) > sum(clean["Favored Prev - Underdog Curr Profit %"]):
                        clean = clean.drop(columns=['Favored Prev - Underdog Curr Profit %','Favored Prev - Underdog Curr Profit % Favored Bet','Favored Prev - Underdog Curr Profit % Underdog Bet'])
                        Underdog_Prev_Favored_Curr_Profit = clean
                        
                        print(Underdog_Prev_Favored_Curr_Profit)
                        
                        dfi.export(Underdog_Prev_Favored_Curr_Profit.T, '/Users/lolatlogan/Downloads/SB/Backdoor Hedging/'+str(datetime.now())+'.png')
                        
                        Html_file = open("/Users/lolatlogan/Public/index.html","w")
                        Html_file.write(str(Underdog_Prev_Favored_Curr_Profit.to_html()))
                        Html_file.close()
                        ! firebase deploy
                    
                    else: 
                        print("Error Compiling")

            ######################################################################################
            # Resetting and Keeping Last Record
            ######################################################################################
            #Getting the Last Date from Each Game and Returnign the Initial DF Back to Normal

            originatableBEFOREdropTail = iterabledf    
            result = iterabledf.sort_values('Time Pulled (CST)').groupby(['Team 1','Team 2','Sport']).tail(1)
            iterabledf = result.drop(columns=['diffs',"diffs2"])
            TotalTime = ((time.time() - start_time)) + TotalTime
            
            if x % 120 == 0:
                print(datetime.now().time())
                try:
                    print("--- %s second current runtime ---" % (time.time() - start_time))
                    print("--- %s average seconds / iteration ---"  % (TotalTime / x))
                    print("--- %s minute total runtime ---"  % (TotalTime/60 ))
                except:
                    pass
            
            time.sleep(1)

    except:
        
        iterabledf = []
        iterabledf = pd.DataFrame(columns=["Team 1","Team 2", "Team 1 Money Line", "Team 2 Money Line","Sport","Time Pulled (CST)"])
        iterabledf = iterabledf.set_index(['Team 1','Team 2','Sport','Time Pulled (CST)'])

        profitablelog = []
        profitablelog = pd.DataFrame(columns=["Team 1","Team 2","Sport","Time Pulled (CST)","Gambit Prev Reward 1",
        "Gambit Prev Reward 2",
        "Gambit Curr Reward 1",
        "Gambit Curr Reward 2",
        "Tokens to Hedge",
        "Favored Prev - Underdog Curr Profit %",
        "Underdog Prev - Favored Curr Profit %",
        "Favored Prev - Underdog Curr Profit % Favored Bet",
        "Favored Prev - Underdog Curr Profit % Underdog Bet",
        "Underdog Prev - Favored Curr Profit % Underdog Bet",
        "Underdog Prev - Favored Curr Profit % Favored Bet"])
        profitablelog = profitablelog.set_index(["Team 1","Team 2","Sport", "Time Pulled (CST)"])

        TotalTime = 0

        #############################################################################################################
        # Gambit
        #############################################################################################################
        start_date = datetime.now()

        GambitProfit = requests.get("https://api.gambitprofit.com/gambit-plays?_limit=200&_sort=PlayDate:DESC").json()
        GambitProfitAPI = pd.DataFrame.from_dict(json.loads(json.dumps(GambitProfit)))[['PlayDate','Team1','Team2',"Calc"]]

        GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team1'].astype(str)
        GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team2'].astype(str)

        GambitProfitAPI['Team 1 Name'] = GambitProfitAPI['Team 1 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()
        GambitProfitAPI['Team 2 Name'] = GambitProfitAPI['Team 2 Name'].str.extract(r"'Name': '(.*?)\'", expand=False).str.strip()

        GambitProfitAPI['PlayDate'] = pd.to_datetime(GambitProfitAPI['PlayDate'], 
                                               format='%Y-%m-%dT%H:%M:%S.000Z', 
                                               errors='coerce') - timedelta(hours=6, minutes=0)

        GambitProfitAPI = GambitProfitAPI.drop(columns=['Team1','Team2'])
        after_start_date = GambitProfitAPI["PlayDate"] >= start_date
        GambitProfitAPI = GambitProfitAPI.loc[after_start_date]

        GambitProfitAPI["Calc"] = GambitProfitAPI["Calc"].astype(str)
        GambitProfitAPI["Profitable"] = GambitProfitAPI["Calc"].str.extract(r"Profitable': (.{3})")[0]         

        GambitProfitAPI = GambitProfitAPI.drop(columns=['Calc','Profitable'])

        GambitProfitAPI['Source'] = "Gambit"

        Team1List = list(GambitProfitAPI['Team 1 Name'])
        Team2List = list(GambitProfitAPI['Team 2 Name'])
        TotalList = Team1List + Team2List
        TotalList = [x for x in TotalList if str(x) != 'nan']


        #############################################################################################################
        # Intertops
        #############################################################################################################

        urldelta = '&delta=1440'

        #############################################################################################################
        # Basketbell
        #############################################################################################################

        ######################################################################################
        # NBA Odds - Moneyline - Single Match
        ######################################################################################
        basketball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=5' + urldelta)
        bs_content = BeautifulSoup(basketball.content, "lxml")
        nba = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NBA Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

        teams = []
        odds = []

        for m in nba:
            for line in m:
                teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                odds.append(re.findall('o="(.*?)"', str(m)))

        teams = teams[0]
        odds = odds[0]

        N = 2
        teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
        teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

        oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
        oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

        nbaoddsdf = pd.concat([teamdf, oddsdf], axis=1)
        nbaoddsdf['Sport'] = "NBA"    


        ######################################################################################
        # NCAAB - Moneyline - Single Match
        ######################################################################################
        bs_content = BeautifulSoup(basketball.content, "lxml")
        ncaab = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NCAAB Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

        teams = []
        odds = []

        for m in ncaab:
            for line in m:
                teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                odds.append(re.findall('o="(.*?)"', str(m)))

        teams = teams[0]
        odds = odds[0]

        N = 2
        teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
        teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

        oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
        oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

        ncaaboddsdf = pd.concat([teamdf, oddsdf], axis=1)
        ncaaboddsdf['Sport'] = "NCAAB"    

        #############################################################################################################
        # Hockey
        #############################################################################################################

        ######################################################################################
        # NHL Odds - Moneyline - Single Match
        ######################################################################################
        icehockey = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=16'+ urldelta)
        bs_content = BeautifulSoup(icehockey.content, "lxml")
        nhl = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"NHL Lines"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

        teams = []
        odds = []

        for m in nhl:
            for line in m:
                teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                odds.append(re.findall('o="(.*?)"', str(m)))

        teams = teams[0]
        odds = odds[0]

        N = 2
        teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
        teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

        oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
        oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

        nhloddsdf = pd.concat([teamdf, oddsdf], axis=1)
        nhloddsdf['Sport'] = "NHL"    



        #############################################################################################################
        # Baseball
        #############################################################################################################

        ######################################################################################
        # MLB Odds 'Spring Training' - Moneyline - Single Match
        ######################################################################################
        baseball = requests.get('http://xmlfeed.intertops.com/xmloddsfeed/v2/xml/feed.ashx?apikey=641af124-9174-eb11-8ad8-003048dd52d5&sportId=4' + urldelta)
        bs_content = BeautifulSoup(baseball.content, "lxml")
        mlbst = BeautifulSoup(str(BeautifulSoup(str(BeautifulSoup(str(bs_content.find_all("cat",{"n":"MLB Spring Training"}))).find_all("m"))).find_all("t",{"n":"Single Match"})))

        teams = []
        odds = []

        for m in mlbst:
            for line in m:
                teams.append(re.findall('>([a-zA-Z].*?)<', str(line)))
                odds.append(re.findall('o="(.*?)"', str(m)))

        teams = teams[0]
        odds = odds[0]

        N = 2
        teamclean = [teams[n:n+N] for n in range(0, len(teams), N)]
        teamdf = pd.DataFrame(teamclean, columns=['Team 1', 'Team 2'])

        oddsclean = [odds[n:n+N] for n in range(0, len(odds), N)]
        oddsdf = pd.DataFrame(oddsclean, columns=['Team 1 Money Line', 'Team 2 Money Line'])

        mlbstoddsdf = pd.concat([teamdf, oddsdf], axis=1)
        mlbstoddsdf['Sport'] = "MLB Spring Training"    
        
        ######################################################################################
        # Consolidate and Append the Old Table
        ######################################################################################
        allsportsoddsdf = nbaoddsdf.append([ncaaboddsdf, nhloddsdf,mlbstoddsdf])

        allsportsoddsdf["Time Pulled (CST)"] = pd.Series([datetime.now()] * len(allsportsoddsdf),dtype=pd.StringDtype())
        allsportsoddsdf["Team 1 Money Line"] = allsportsoddsdf["Team 1 Money Line"].astype(float)
        allsportsoddsdf["Team 2 Money Line"] = allsportsoddsdf["Team 2 Money Line"].astype(float)

        ######################################################################################
        # Filtering ESPN / Dratings Table by Gambit API
        ######################################################################################

        allsportsoddsdf['Team 1'] = allsportsoddsdf['Team 1'].str.replace('&amp;','&')
        allsportsoddsdf['Team 2'] = allsportsoddsdf['Team 2'].str.replace('&amp;','&')

        allsportsoddsdf['Team 1 in API'] = allsportsoddsdf['Team 1'].apply(lambda x: ','.join([i for i in TotalList if i in x]))
        allsportsoddsdf['Team 2 in API'] = allsportsoddsdf['Team 2'].apply(lambda x: ','.join([i for i in TotalList if i in x]))

        allsportsoddsdf = allsportsoddsdf[(allsportsoddsdf['Team 1 in API'] != '') | (allsportsoddsdf['Team 2 in API'] != '')]
        allsportsoddsdf = allsportsoddsdf.drop(columns=['Team 1 in API','Team 2 in API'])

        ######################################################################################
        # Appending the New Data to the Old Data 
        ######################################################################################
        allsportsoddsdf = allsportsoddsdf.set_index(['Team 1','Team 2','Sport','Time Pulled (CST)'])
        iterabledf = iterabledf.append(allsportsoddsdf,sort=True)
        iterabledf = iterabledf.sort_index(ascending=True)

        # print(allsportsoddsdf,'\n',iterabledf)
        iterabledf

        time.sleep(30)
        print("Something Broke - Sleeping for 30 Seconds, Resettingm and Then Going Again")

                                                                             Team 1 Money Line  \
Team 1               Team 2                Sport Time Pulled (CST)                               
Edmonton Oilers      Calgary Flames        NHL   2021-03-15 17:30:34.464268             2.0500   
Indiana Pacers       Denver Nuggets        NBA   2021-03-15 17:30:34.464268             2.6000   
Los Angeles Clippers Dallas Mavericks      NBA   2021-03-15 17:30:34.464268             1.8000   
Los Angeles Lakers   Golden State Warriors NBA   2021-03-15 17:30:34.464268             1.8000   
Memphis Grizzlies    Phoenix Suns          NBA   2021-03-15 17:30:34.464268             3.3500   
Montreal Canadiens   Winnipeg Jets         NHL   2021-03-15 17:30:34.464268             1.8696   
New York Knicks      Brooklyn Nets         NBA   2021-03-15 17:30:34.464268             3.8000   
San Antonio Spurs    Detroit Pistons       NBA   2021-03-15 17:30:34.464268             1.7692   
San Jose Sharks     